# Analyzing climate change effects on Madagascar

In this jupyter notebook, we aim to analyze the climate change effects on Madagascar through the following research questions:
- What are significant trends over the past 20 years in the precipitation, temperature, and humidity in Madagascar? 
- How have these significant trends affected the livelihood of Madagascar? This is including but not limited to: the health of its inhabitants, natural disasters such as flooding, food security, and biodiversity levels. 


## Overview of datasets:
- First dataset (Climate change Madagascar): [https://www.kaggle.com/brsdincer/climate-change-madagascar-turkey-nasa](https://www.kaggle.com/brsdincer/climate-change-madagascar-turkey-nasa)
- Second dataset (Climate historical data): [https://climateknowledgeportal.worldbank.org/country/madagascar/climate-data-historical](https://climateknowledgeportal.worldbank.org/country/madagascar/climate-data-historical) Note that we might not use this dataset
- Third dataset (Climate change indicators): [https://data.humdata.org/dataset/world-bank-climate-change-indicators-for-madagascar](https://data.humdata.org/dataset/world-bank-climate-change-indicators-for-madagascar)

In [1]:
# Install new packages, need to uncomment this for the first time
#pip install netCDF4

In [2]:
# import the required libraries
import numpy as np
import pandas as pd
import netCDF4 as nc

Helpful link dealing with nc4 data: https://towardsdatascience.com/read-netcdf-data-with-python-901f7ff61648

In [7]:
# Read in the first dataset
file_path_header = "data/firstDataset/AUG_1_2021_CLIMATE_CHANGE_COMPARING/MADAGASCAR/2021_AUG_1/"
file_names = ["tavg1_2d_flx_Nx-202108251432output.10532.webform.nc4", "tavg1_2d_lnd_Nx-202108251427output.8348.webform.nc4", "tavg1_2d_ocn_Nx-202108251420output.5096.webform.nc4", "tavg1_2d_rad_Nx-202108251429output.8623.webform.nc4", "tavg1_2d_slv_Nx-202108251418output.3443.webform.nc4", "tavg3_2d_aer_Nx-202108251425output.5858.webform.nc4", "tavg3_2d_chm_Nx-202108251422output.5477.webform.nc4"]
list_ds = []
for file_name in file_names:
    list_ds.append(nc.Dataset(file_path_header + file_name))
print(list_ds)


[<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: COARDS
    calendar: standard
    comments: File
    model: geos/das
    center: gsfc
    dimensions(sizes): time(24), longitude(33), latitude(63)
    variables(dimensions): float64 time(time), float64 longitude(longitude), float64 latitude(latitude), float32 rhoa(time, latitude, longitude)
    groups: , <class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: COARDS
    calendar: standard
    comments: File
    model: geos/das
    center: gsfc
    dimensions(sizes): time(24), longitude(33), latitude(63)
    variables(dimensions): float64 time(time), float64 longitude(longitude), float64 latitude(latitude), float32 prectot(time, latitude, longitude), float32 prmc(time, latitude, longitude), float32 grn(time, latitude, longitude), float32 evland(time, latitude, longitude), float32 teland(time, latitude, longitude), float32 evpintr(time, la

In [13]:
# Read in the second Dataset- appears to not really be a way this data is sorted, so we might not want to use it
with open("data/ctmp6190/ctmp6190.dat") as f:
    for line in f:
        print(line)
x = np.fromfile("data/ctmp6190/ctmp6190.dat")
print(x)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  273  259  269  277  277  276  274  274  275  272  268  269  266  268  265  272  271  271  265  257  249  245  211  225  237  237  240  239  236  234  233  237  251  260  265  266  266  265  263  260  258  256  254  253  255  253  252  251  251  252  254  254  261  268  273  274  278  283  283  283  285  284  284  284  282  282  278  269  262  256  218  185  216  198  211  187  200  155  129  122  172  220  241  258  277  272  269  257  266  262  272  274  279  281  285  286  286  289  296-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999  281  269  231  269  283-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999  277-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999-9999  277

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
# Read in third dataset
climate_change_indicators = pd.read_csv("data/climate-change_mdg.csv")
climate_change_indicators.head()

,Country Name,Country ISO3,Year,Indicator Name,Indicator Code,Value
0,#country+name,#country+code,#date+year,#indicator+name,#indicator+code,#indicator+value+num
1,Madagascar,MDG,2018,Agricultural land (sq. km),AG.LND.AGRI.K2,408950
2,Madagascar,MDG,2017,Agricultural land (sq. km),AG.LND.AGRI.K2,408950
3,Madagascar,MDG,2016,Agricultural land (sq. km),AG.LND.AGRI.K2,408950
4,Madagascar,MDG,2015,Agricultural land (sq. km),AG.LND.AGRI.K2,408950


In [19]:
# Read in the fourth dataset about food security
food_security = pd.read_csv("data/food_security/pblstatssum-2.csv")
food_security.head()

,SvyDate,ADM0_NAME,ADM1_NAME,ADM2_NAME,AdminStrata,Variable,VariableDescription,Demographic,Mean
0,#date,#country+name,#adm1+name,#adm2+name,#loc+name,#indicator+code,#indicator+name,#category,#indicator+value+num
1,2016-05-01T00:00:00,Madagascar,NaN,NaN,Analamanga,BorrowOrHelp,# of days household using this coping strategy...,NaN,1.15652
2,2016-05-01T00:00:00,Madagascar,NaN,NaN,Androy,BorrowOrHelp,# of days household using this coping strategy...,NaN,2.34211
3,2016-05-01T00:00:00,Madagascar,NaN,NaN,Anosy,BorrowOrHelp,# of days household using this coping strategy...,NaN,1.7561
4,2016-05-01T00:00:00,Madagascar,NaN,NaN,Atsinanana,BorrowOrHelp,# of days household using this coping strategy...,NaN,1.49296


# Other datasets and factors
From the links, [https://madagascar.opendataforafrica.org/tkkbtv/world-development-indicators-wdi-november-2014](https://madagascar.opendataforafrica.org/tkkbtv/world-development-indicators-wdi-november-2014) and [https://www.gbif.org/occurrence/search?country=MG](https://www.gbif.org/occurrence/search?country=MG), we can filter for different datasets based on Madagascar that we decide we want to use